### Week 6, Day 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel, set_default_openai_client, set_tracing_export_api_key
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown
from openai import AsyncOpenAI, AsyncAzureOpenAI
import os

load_dotenv(override=True)

True

In [15]:
from accounts import Account
openai = os.getenv("OPENAI_API_KEY")
openai_client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-12-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment="gtp-4o-mini"
)

gemini_client = AsyncOpenAI(base_url=os.getenv('GEMINI_API_BASE'), api_key=os.getenv('GEMINI_API_KEY'))
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.5-flash",openai_client=gemini_client)

set_default_openai_client(gemini_client);
set_tracing_export_api_key(openai);

In [6]:
account = Account.get("Adrian")
account

Account(name='adrian', balance=9837.676, strategy='', holdings={'AMZN': 3}, transactions=[3 shares of AMZN at 54.108 each.], portfolio_value_time_series=[('2025-08-21 17:58:34', 10134.676)])

In [5]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "adrian", "balance": 9837.676, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 54.108, "timestamp": "2025-08-21 17:58:34", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-21 17:58:34", 10134.676]], "total_portfolio_value": 10134.676, "total_profit_loss": 134.67599999999948}'

In [7]:
account.report()

'{"name": "adrian", "balance": 9837.676, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 54.108, "timestamp": "2025-08-21 17:58:34", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-21 17:58:34", 10134.676], ["2025-08-21 17:58:50", 9948.676]], "total_portfolio_value": 9948.676, "total_profit_loss": -51.324000000000524}'

In [8]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 54.108,
  'timestamp': '2025-08-21 17:58:34',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [9]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.session.list_tools()


In [10]:
mcp_tools.tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

In [13]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Adrian and my account is under the name Adrian. What's my balance and my holdings?"
model = gemini_model

In [16]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Your cash balance is $9837.68. You are holding 3 shares of AMZN.

### Now let's build our own MCP Client

In [17]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None), Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None), Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', 'ty

In [18]:
request = "My name is Adrian and my account is under the name Adrian. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

You have $9837.68 in your account.

In [19]:
context = await read_accounts_resource("Adrian")
print(context)

{"name": "adrian", "balance": 9837.676, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 54.108, "timestamp": "2025-08-21 17:58:34", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-21 17:58:34", 10134.676], ["2025-08-21 17:58:50", 9948.676], ["2025-08-21 18:45:45", 10077.676]], "total_portfolio_value": 10077.676, "total_profit_loss": 77.67599999999948}


In [20]:
from accounts import Account
Account.get("Adrian").report()

'{"name": "adrian", "balance": 9837.676, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 54.108, "timestamp": "2025-08-21 17:58:34", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-08-21 17:58:34", 10134.676], ["2025-08-21 17:58:50", 9948.676], ["2025-08-21 18:45:45", 10077.676], ["2025-08-21 18:45:58", 10038.676]], "total_portfolio_value": 10038.676, "total_profit_loss": 38.675999999999476}'

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Make your own MCP Server! Make a simple function to return the current Date, and expose it as a tool so that an Agent can tell you today's date.<br/>Harder optional exercise: then make an MCP Client, and use a native OpenAI call (without the Agents SDK) to use your tool via your client.
            </span>
        </td>
    </tr>
</table>